In [1]:
import numpy as np
import pandas as pd
import time
from functools import partial

from scipy.stats import randint, uniform
from sim_lib import simulation

pd.options.mode.chained_assignment = None


In [2]:
def eps_greedy(history: pd.DataFrame, eps: float):
    if uniform.rvs() < eps:
        n = history.shape[0]
        return history.index[randint.rvs(0, n)]

    ctr = history['clicks'] / (history['impressions'] + 10)
    n = np.argmax(ctr)
    return history.index[n]

policy = partial(eps_greedy, eps=0.06)

In [2]:
# seed for homework
seed = 18475
np.random.seed(seed=seed)

start = time.time()
output = simulation(policy, n=200000, seed=seed)
end = time.time()
end - start

In [4]:
# baseline regret
output['regret'], output['regret']/output['rounds'],  output['total_banners']

(1540.7609683932544, 0.007703804841966272, 184)

Попробуем алгоритм UCB

In [3]:
class UCB():
    def __init__(self, c=1):
        self.t = 0
        self.c = c # exploration/exploitation balance
        
    def __call__(self, history):
        self.t += 1
        exploitation = history['clicks'] / (history['impressions'] + 1) 
        exploration = np.sqrt(2 * np.log(self.t) / (history['impressions'] + 1))
        i = np.argmax(exploitation + self.c * exploration)
        return history.index[i]

Подберем оптимальное `с`

In [4]:
C = [1, 0.1, 0.01, 0.001, 0.0001]
regrets = []
for c in C:
    policy = UCB(c)
    output = simulation(policy, n=50000, seed=seed)
    regrets.append(output['regret'])
n = np.argmin(regrets)
print(regrets)
print(f'Best c: {C[n]}')

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
[2393.717146557163, 76.03125965508134, 1844.4500083428345, 225.1078906

Попробуем уточнить значение оптимума

In [5]:
C = [0.4, 0.2, 0.1, 0.05]
regrets = []
for c in C:
    policy = UCB(c)
    output = simulation(policy, n=100000, seed=seed)
    regrets.append(output['regret'])
n = np.argmin(regrets)
print(regrets)
print(f'Best c: {C[n]}')

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have b

Протестируем на 200 тыс. итераций

In [6]:
C = [0.07, 0.06, 0.05, 0.04]
regrets = []
for c in C:
    policy = UCB(c)
    output = simulation(policy, n=200000, seed=seed)
    regrets.append(output['regret'])

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impr

Результат

In [7]:
n = np.argmin(regrets)
print(f'Best c: {C[n]}')
print(f'Best regret: {regrets[n]}')

Best c: 0.06
Best regret: 45.989836615452276
